In [1]:
import csv
from csv import DictReader
import requests
import json


Relation types are listed at https://support.datacite.org/docs/eventdata-guide#relation-type-id. This script looks for the resources (subjects) that are related somehow to each dataset PID (object) in the given CSV or TXT file.

I'm ignoring the following relation types since I'm fairly confident that publishers aren't using these to describe the relationship between two different studies:
- is-version-of
- is-new-version-of
- is-previous-version-of
- is-part-of
- has-part
- is-identical-to

In [2]:
relationTypes = [
    'Is-cited-by',
    'cites',
    'is-supplement-to',
    'is-supplemented-by',
    'is-referenced-by',
    'references'
]

In [8]:
# # Enter path to CSV or text file containing list of dataset PIDs
# datasetPIDFile = 'dataset_pids_QDR Main Collection_2021.08.05_18.55.01.csv'
#
# # Read in list of dataset PIDs from given CSV or text file
# datasetPIDs = []
#
# if '.csv' in datasetPIDFile:
#     with open(datasetPIDFile, mode='r', encoding='utf-8') as f:
#         csvDictReader = DictReader(f, delimiter=',')
#         for row in csvDictReader:
#             datasetPIDs.append(row['persistent_id'].rstrip())
# elif 'txt' in datasetPIDFile:
#     datasetPIDFile = open(datasetPIDFile, 'r')
#     for datasetPID in datasetPIDFile:
#         datasetPIDs.append(datasetPID.rstrip())

In [19]:
print(len(datasetPIDs))

1


In [14]:
# Create CSV file for writing data requested from DataCite API
# citationsOutputFile = 'citations_of_nih_funded_datasets.csv'
#
# with open(citationsOutputFile, mode='w') as opencsvfile:
#     opencsvfile = csv.writer(opencsvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     opencsvfile.writerow(['subject', 'relation', 'object (dataset)', 'occured_at', 'timestamp'])

In [27]:
datasetPIDs = [
    'doi:10.7910/DVN/YETJIS']

# Create variables for keeping count of script's progress
pidCount = 1
pidTotal = len(datasetPIDs)

dataciteEventsAPI = 'https://api.datacite.org/events'
for pid in datasetPIDs:
    params = {
        'doi': pid.replace('doi:', ''),
        'page[number]': 1
    }
    response = requests.get(dataciteEventsAPI, params=params)
    relationData = response.json()
    print(json.dumps(relationData, indent=4))
#     print('Searching for metrics for %s (%s of %s)' %(pid, pidCount, pidTotal))

{
    "data": [],
    "meta": {
        "total": 0,
        "total-pages": 0,
        "page": 1
    },
    "links": {
        "self": "https://api.datacite.org/events?doi=10.7910%2FDVN%2FYETJIS&page%5Bnumber%5D=1"
    }
}


In [26]:
totalPages = relationData['meta']['total-pages']
print(totalPages)

1


In [ ]:
    if len(relationData['data']) > 0:
        totalPages = relationData['meta']['total-pages']

        # Initialization for paginating through Search API results and showing progress
#         pageNumber = 1
#         pageSize = 10
        condition = True
        relationCount = 0

        while condition:
#             params = params
            response = requests.get(dataciteEventsAPI)
            relationData = response.json()
            for i in relationData['data']:
                if i['attributes']['relation-type-id'] in relationships:
                    subject = i['attributes']['subj-id']
                    relationTypeId = i['attributes']['relation-type-id']
                    object = i['attributes']['obj-id']
                    occurredAt = i['attributes']['occurred-at']
                    timeStamp = i['attributes']['timestamp']
                    
                    print(subject, relationTypeId, object, occurredAt, timeStamp)
                    
                    # Write values of the three variables to a new row in the CSV
#                     with open('data.csv', mode='a') as datasets:
#                         datasets = csv.writer(datasets, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                         datasets.writerow([subject, relationTypeId, object, occurredAt, timeStamp])
                    
                    relationCount += 1

            params['page[number]'] += 1
            condition = params['page[number]'] <= totalPages
    else:
        print('No relation data found')
    pidCount += 1